In [1]:
import pandas as pd
import numpy as np

In [2]:
numeric = "../input_orig/train_numeric.csv"
date = "../input_orig/train_date.csv"
categorical = "../input_orig/train_categorical.csv"

numeric_te = "../input_orig/test_numeric.csv"
date_te = "../input_orig/test_date.csv"
categorical_te = "../input_orig/test_categorical.csv"

In [3]:
def orgainize(features):
    line_features = {}
    station_features = {}
    lines = set([f.split('_')[0] for f in features])
    stations = set([f.split('_')[1] for f in features])
    
    for l in lines:
        line_features[l] = [f for f in features if l+'_' in f]
        
    for s in stations:
        station_features[s] = [f for f in features if s+'_' in f]
            
    return line_features, station_features

In [5]:
features_num = pd.read_csv(numeric, nrows=1).drop(['Response', 'Id'], axis=1).columns.values
line_features_num, station_features_num = orgainize(features_num)
print("Features in Station 32: {}".format( station_features_num['S32'] ))

Features in Station 32: ['L3_S32_F3850']


In [6]:
features_date = pd.read_csv(date, nrows=1).drop(['Id'], axis=1).columns.values
line_features_date, station_features_date = orgainize(features_date)
print("Features in Station 32: {}".format( station_features_date['S32'] ))

Features in Station 32: ['L3_S32_D3852']


In [7]:
features_cat = pd.read_csv(categorical, nrows=1).drop(['Id'], axis=1).columns.values
line_features_cat, station_features_cat = orgainize(features_cat)
print("Features in Station 32: {}".format( station_features_cat['S32'] ))

Features in Station 32: ['L3_S32_F3851', 'L3_S32_F3853', 'L3_S32_F3854']


In [8]:
CHUNKSIZE = 100000
NROWS_TR = 1183747
nrows_tr = 0
NROWS_TE = 1183748
nrows_te = 0
ID_COLUMN = 'Id'
TARGET_COLUMN = 'Response'
SEED = 0

def make_features_num(df):
    new_features = pd.DataFrame({})
    for s in station_features_num.keys():
        station_data = df[station_features_num[s]]
        col = s+'_max_num'
        new_features[col] = station_data.max(axis=1)
        col = s+'_min_num'
        new_features[col] = station_data.min(axis=1)
    return new_features

tr_station_minmax_num = pd.DataFrame()
for tr in pd.read_csv(numeric, chunksize=CHUNKSIZE, dtype=np.float32):
    feats = np.setdiff1d(tr.columns, [ID_COLUMN, TARGET_COLUMN])
    chunk_features = make_features_num(tr[feats])
    tr_station_minmax_num = tr_station_minmax_num.append(chunk_features)
    
    nrows_tr += CHUNKSIZE
    if nrows_tr >= NROWS_TR:
        break
        
te_station_minmax_num = pd.DataFrame()
for te in pd.read_csv(numeric_te, chunksize=CHUNKSIZE, dtype=np.float32):
    feats = np.setdiff1d(te.columns, [ID_COLUMN, TARGET_COLUMN])
    chunk_features = make_features_num(te[feats])
    te_station_minmax_num = te_station_minmax_num.append(chunk_features)
    
    nrows_te += CHUNKSIZE
    if nrows_te >= NROWS_TE:
        break

In [9]:
tr_station_minmax_num.head()

,S17_max_num,S17_min_num,S48_max_num,S48_min_num,S23_max_num,S23_min_num,S10_max_num,S10_min_num,S18_max_num,S18_min_num,...,S32_max_num,S32_min_num,S26_max_num,S26_min_num,S7_max_num,S7_min_num,S30_max_num,S30_min_num,S33_max_num,S33_min_num
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.060,-0.164,0.275,-0.177,0.026,-0.119
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.315,-0.046,...,NaN,NaN,NaN,NaN,NaN,NaN,0.255,-0.143,0.000,-0.210
2,NaN,NaN,NaN,NaN,NaN,NaN,0.105,-0.047,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.134,-0.221,0.236,0.000
3,NaN,NaN,NaN,NaN,NaN,NaN,0.139,-0.049,NaN,NaN,...,NaN,NaN,NaN,NaN,0.402,-0.164,0.164,-0.173,0.165,0.000
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.014,-0.164,0.137,-0.186,0.147,-0.074


In [10]:
te_station_minmax_num.head()

,S17_max_num,S17_min_num,S48_max_num,S48_min_num,S23_max_num,S23_min_num,S10_max_num,S10_min_num,S18_max_num,S18_min_num,...,S32_max_num,S32_min_num,S26_max_num,S26_min_num,S7_max_num,S7_min_num,S30_max_num,S30_min_num,S33_max_num,S33_min_num
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.131,-0.251,0.027,-0.256
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.133,-0.230,0.062,-0.193
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.336,-0.064,NaN,NaN,0.181,-0.161,0.167,-0.119
3,NaN,NaN,NaN,NaN,NaN,NaN,0.106,-0.194,NaN,NaN,...,NaN,NaN,0.177,-0.611,0.401,-0.381,0.112,-0.151,0.000,-0.073
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.200,-0.125,NaN,NaN,0.296,-0.199,0.124,-0.119


In [11]:
tr_station_minmax_num.shape, te_station_minmax_num.shape

((1183747, 100), (1183748, 100))

In [12]:
CHUNKSIZE = 100000
NROWS_TR = 1183747
nrows_tr = 0
NROWS_TE = 1183748
nrows_te = 0
ID_COLUMN = 'Id'
TARGET_COLUMN = 'Response'
SEED = 0

def make_features_date(df):
    new_features = pd.DataFrame({})
    for s in station_features_date.keys():
        station_data = df[station_features_date[s]]
        col = s+'_max_date'
        new_features[col] = station_data.max(axis=1)
        col = s+'_min_date'
        new_features[col] = station_data.min(axis=1)
    return new_features

tr_station_minmax_date = pd.DataFrame()
for tr in pd.read_csv(date, chunksize=CHUNKSIZE, dtype=np.float32):
    feats = np.setdiff1d(tr.columns, [ID_COLUMN])
    chunk_features = make_features_date(tr[feats])
    tr_station_minmax_date = tr_station_minmax_date.append(chunk_features)
    
    nrows_tr += CHUNKSIZE
    if nrows_tr >= NROWS_TR:
        break
        
te_station_minmax_date = pd.DataFrame()
for te in pd.read_csv(date_te, chunksize=CHUNKSIZE, dtype=np.float32):
    feats = np.setdiff1d(te.columns, [ID_COLUMN])
    chunk_features = make_features_date(te[feats])
    te_station_minmax_date = te_station_minmax_date.append(chunk_features)
    
    nrows_te += CHUNKSIZE
    if nrows_te >= NROWS_TE:
        break

In [13]:
tr_station_minmax_date.head()

,S17_max_date,S17_min_date,S48_max_date,S48_min_date,S23_max_date,S23_min_date,S10_max_date,S10_min_date,S18_max_date,S18_min_date,...,S32_max_date,S32_min_date,S26_max_date,S26_min_date,S7_max_date,S7_min_date,S30_max_date,S30_min_date,S33_max_date,S33_min_date
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,82.260002,82.260002,87.260002,87.260002,87.279999,87.279999
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1313.150024,1313.150024,...,NaN,NaN,NaN,NaN,NaN,NaN,1315.729980,1315.729980,1315.750000,1315.750000
2,NaN,NaN,NaN,NaN,NaN,NaN,1618.729980,1618.729980,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1624.410034,1624.410034,1624.420044,1624.420044
3,NaN,NaN,NaN,NaN,NaN,NaN,1149.219971,1149.219971,NaN,NaN,...,NaN,NaN,NaN,NaN,1149.219971,1149.219971,1154.130005,1154.130005,1154.140015,1154.140015
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,602.669983,602.669983,606.000000,606.000000,606.010010,606.010010


In [14]:
te_station_minmax_date.head()

,S17_max_date,S17_min_date,S48_max_date,S48_min_date,S23_max_date,S23_min_date,S10_max_date,S10_min_date,S18_max_date,S18_min_date,...,S32_max_date,S32_min_date,S26_max_date,S26_min_date,S7_max_date,S7_min_date,S30_max_date,S30_min_date,S33_max_date,S33_min_date
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,800.659973,800.659973,800.679993,800.679993
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1060.040039,1060.040039,1060.060059,1060.060059
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,704.109985,704.109985,NaN,NaN,710.950012,710.950012,711.070007,711.070007
3,NaN,NaN,NaN,NaN,NaN,NaN,255.490005,255.490005,NaN,NaN,...,NaN,NaN,255.500000,255.500000,255.479996,255.479996,256.250000,256.250000,256.269989,256.269989
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,760.929993,760.929993,NaN,NaN,770.270020,770.270020,770.280029,770.280029


In [15]:
tr_station_minmax_date.shape, te_station_minmax_date.shape

((1183747, 104), (1183748, 104))

In [16]:
tr_station_minmax = pd.concat([tr_station_minmax_num,tr_station_minmax_date],axis=1)
te_station_minmax = pd.concat([te_station_minmax_num,te_station_minmax_date],axis=1)

In [16]:
#tr_station_sum.to_hdf('tr_station_sum7.hdf5','table')
#te_station_sum.to_hdf('te_station_sum7.hdf5','table')

In [17]:
CHUNKSIZE = 50000
NROWS_TR = 1183747
nrows_tr = 0
NROWS_TE = 1183748
nrows_te = 0
ID_COLUMN = 'Id'
TARGET_COLUMN = 'Response'
SEED = 0

def make_features_num(df):
    new_features = pd.DataFrame({})
    for s in line_features_num.keys():
        line_data = df[line_features_num[s]]
        col = s+'_max_num'
        new_features[col] = line_data.max(axis=1)
        col = s+'_min_num'
        new_features[col] = line_data.min(axis=1)
    return new_features

tr_line_minmax_num = pd.DataFrame()
for tr in pd.read_csv(numeric, chunksize=CHUNKSIZE, dtype=np.float32):
    feats = np.setdiff1d(tr.columns, [ID_COLUMN, TARGET_COLUMN])
    chunk_features = make_features_num(tr[feats])
    tr_line_minmax_num = tr_line_minmax_num.append(chunk_features)
    
    nrows_tr += CHUNKSIZE
    if nrows_tr >= NROWS_TR:
        break
        
te_line_minmax_num = pd.DataFrame()
for te in pd.read_csv(numeric_te, chunksize=CHUNKSIZE, dtype=np.float32):
    feats = np.setdiff1d(te.columns, [ID_COLUMN, TARGET_COLUMN])
    chunk_features = make_features_num(te[feats])
    te_line_minmax_num = te_line_minmax_num.append(chunk_features)
    
    nrows_te += CHUNKSIZE
    if nrows_te >= NROWS_TE:
        break

In [18]:
CHUNKSIZE = 50000
NROWS_TR = 1183747
nrows_tr = 0
NROWS_TE = 1183748
nrows_te = 0
ID_COLUMN = 'Id'
TARGET_COLUMN = 'Response'
SEED = 0

def make_features_date(df):
    new_features = pd.DataFrame({})
    for s in line_features_date.keys():
        line_data = df[line_features_date[s]]
        col = s+'_max_date'
        new_features[col] = line_data.max(axis=1)
        col = s+'_min_date'
        new_features[col] = line_data.min(axis=1)
    return new_features

tr_line_minmax_date = pd.DataFrame()
for tr in pd.read_csv(date, chunksize=CHUNKSIZE, dtype=np.float32):
    feats = np.setdiff1d(tr.columns, [ID_COLUMN])
    chunk_features = make_features_date(tr[feats])
    tr_line_minmax_date = tr_line_minmax_date.append(chunk_features)
    
    nrows_tr += CHUNKSIZE
    if nrows_tr >= NROWS_TR:
        break
        
te_line_minmax_date = pd.DataFrame()
for te in pd.read_csv(date_te, chunksize=CHUNKSIZE, dtype=np.float32):
    feats = np.setdiff1d(te.columns, [ID_COLUMN])
    chunk_features = make_features_date(te[feats])
    te_line_minmax_date = te_line_minmax_date.append(chunk_features)
    
    nrows_te += CHUNKSIZE
    if nrows_te >= NROWS_TE:
        break

In [19]:
tr_line_minmax = pd.concat([tr_line_minmax_num,tr_line_minmax_date],axis=1)
te_line_minmax = pd.concat([te_line_minmax_num,te_line_minmax_date],axis=1)

#tr_line_sum.to_hdf('tr_line_sum7.hdf5','table')
#te_line_sum.to_hdf('te_line_sum7.hdf5','table')

In [27]:
tr_line_station_minmax = pd.concat([tr_station_minmax,tr_line_minmax],axis=1)
te_line_station_minmax = pd.concat([te_station_minmax,te_line_minmax],axis=1)

In [31]:
te_line_station_minmax.tail()

,S17_max_num,S17_min_num,S48_max_num,S48_min_num,S23_max_num,S23_min_num,S10_max_num,S10_min_num,S18_max_num,S18_min_num,...,L3_max_num,L3_min_num,L0_max_date,L0_min_date,L2_max_date,L2_min_date,L1_max_date,L1_min_date,L3_max_date,L3_min_date
1183743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.704,-0.516,653.869995,653.849976,NaN,NaN,NaN,NaN,655.450012,655.429993
1183744,NaN,NaN,NaN,NaN,NaN,NaN,0.104,-0.044,NaN,NaN,...,0.268,-0.480,907.369995,907.340027,909.130005,909.130005,NaN,NaN,911.289978,911.270020
1183745,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.520,-0.270,185.960007,185.919998,NaN,NaN,NaN,NaN,186.320007,186.289993
1183746,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.520,-0.276,570.890015,570.849976,NaN,NaN,NaN,NaN,572.669983,572.650024
1183747,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.201,-0.480,1412.829956,1412.800049,NaN,NaN,NaN,NaN,1413.180054,1413.160034


In [30]:
tr_line_station_minmax.to_hdf('feats/tr_line_station_minmax8.hdf5','table')
te_line_station_minmax.to_hdf('feats/te_line_station_minmax8.hdf5','table')

In [ ]:
tr_line_station_minmax.shape